## TREND ANALYSIS 
Design and implement a monthly trend‐analysis framework to serve as the foundational model for budget-versus-actual variance calculations. Incorporate clear, concise formulas that can be embedded directly into your dashboard for real-time reporting. Ultimately, package these routines as a programmable calculator module for web deployment—automating recurring computations and freeing analysts from manually recalculating figures.

## Centralizing DATA
Centralizing Data
Historically, unit costs have been calculated by combining materials, labor, and overhead expenses. Because overhead allocations originate at the corporate level, estimating them can be challenging and can introduce variability into the unit‐cost calculation. For this project, we have simplified and standardized these metrics into a single extraction process. Going forward, all company data will be consolidated into monthly query‐based batches for consistency and reliability. For the purposes of this project, we currently focus on dataframe extracts moving forward to automated queries.

In [4]:
## Centralizing DATAset
## 1. Congifuring the labor_data csv and sampe cost_data.xlsx
import pandas as pd

# If your CSV is in a file:
df1 = pd.read_csv('labor_data.csv')
                  
# 1. Ensure the date columns are datetime
df1['date'] = pd.to_datetime(df1['date'])

# 2. Create a “month” key (YYYY-MM) in each
df1['month'] = df1['date'].dt.to_period('M')

df1.head(5)
df1.tail(5)
df1.head(5)
df1.tail(5)


,date,avg_hourly_earnings,month
226,2025-01-01,34.73,2025-01
227,2025-02-01,34.88,2025-02
228,2025-03-01,35.17,2025-03
229,2025-04-01,35.11,2025-04
230,2025-05-01,35.28,2025-05


In [5]:
import pandas as pd

# 2. Configuring the sample cost_data.xlsx
# Load the workbook into a DataFrame
df2 = pd.read_excel('sample_cost_data.xlsx')

# 1. Ensure the date columns are datetime
df2['date'] = pd.to_datetime(df2['date'])

# 2. Create a “month” key (YYYY-MM) in each
df2['month'] = df2['date'].dt.to_period('M')

# Preview the first 5 rows
print(df2.head(5))

# Preview the last 5 rows
print(df2.tail(5))

        date   price commodity    month
0 1990-01-01  1528.0  Aluminum  1990-01
1 1990-02-01  1454.0  Aluminum  1990-02
2 1990-03-01  1567.0  Aluminum  1990-03
3 1990-04-01  1526.0  Aluminum  1990-04
4 1990-05-01  1527.0  Aluminum  1990-05
           date  price                     commodity    month
1652 2024-12-01  100.0  Rare-Earth Metals Import PPI  2024-12
1653 2025-01-01  100.2  Rare-Earth Metals Import PPI  2025-01
1654 2025-02-01   94.4  Rare-Earth Metals Import PPI  2025-02
1655 2025-03-01   96.2  Rare-Earth Metals Import PPI  2025-03
1656 2025-04-01   90.7  Rare-Earth Metals Import PPI  2025-04


In [6]:
# 3. Merge on that month period
merged = pd.merge(
    df1,
    df2,
    on='month',
    how='inner',
)

In [7]:
print(merged)

        date_x  avg_hourly_earnings    month     date_y         price  \
0   2006-03-01                20.71  2006-03 2006-03-01   2432.478261   
1   2006-03-01                20.71  2006-03 2006-03-01   5123.673913   
2   2006-03-01                20.71  2006-03 2006-03-01  14925.478261   
3   2006-03-01                20.71  2006-03 2006-03-01      6.890000   
4   2006-04-01                20.83  2006-04 2006-04-01   2623.858333   
..         ...                  ...      ...        ...           ...   
939 2025-04-01                35.11  2025-04 2025-04-01   9172.695909   
940 2025-04-01                35.11  2025-04 2025-04-01  15146.002273   
941 2025-04-01                35.11  2025-04 2025-04-01      3.420000   
942 2025-04-01                35.11  2025-04 2025-04-01     90.700000   
943 2025-05-01                35.28  2025-05 2025-05-01      3.120000   

                        commodity  
0                        Aluminum  
1                          Copper  
2              

In [9]:
import pandas as pd

# assuming your merged DataFrame is called `df`
df = merged.copy()

# 1. Rename columns for clarity
df = df.rename(columns={
    'date_x': 'earnings_date',
    'date_y': 'price_date',
    'month': 'year_month'
})

# 2. Convert types
# — ensure your dates are true timestamps
df['earnings_date'] = pd.to_datetime(df['earnings_date'])
df['price_date']    = pd.to_datetime(df['price_date'])
# — if you want year_month as a Timestamp instead of Period:
df['year_month']    = df['year_month'].dt.to_timestamp()

# 3. Drop any totally redundant columns
# If price_date and earnings_date always match year_month exactly, you can drop one:
#df = df.drop(columns=['price_date'])

# 4. Reorder columns into a more logical flow
df = df[[
    'year_month',
    'earnings_date',
    'avg_hourly_earnings',
    'price_date',
    'commodity',
    'price'
]]

# 5. (Optional) Remove duplicates, if any slipped in
df = df.drop_duplicates()

# 6. (Optional) Pivot so each commodity becomes its own column of prices
df_pivot = (
    df
    .pivot_table(
        index=['year_month', 'earnings_date', 'avg_hourly_earnings'],
        columns='commodity',
        values='price'
    )
    .reset_index()
)

# 7. (Optional) Sort chronologically
df_pivot = df_pivot.sort_values(by='year_month')

# Now `df` is your tidy long‐form table, and `df_pivot` is wide‐form.
print(df.head())
print(df_pivot.head())
# Now `df` is your tidy long‐form table, and `df_pivot` is wide‐form.
print(df.tail())
print(df_pivot.tail())


  year_month earnings_date  avg_hourly_earnings price_date  \
0 2006-03-01    2006-03-01                20.71 2006-03-01   
1 2006-03-01    2006-03-01                20.71 2006-03-01   
2 2006-03-01    2006-03-01                20.71 2006-03-01   
3 2006-03-01    2006-03-01                20.71 2006-03-01   
4 2006-04-01    2006-04-01                20.83 2006-04-01   

                      commodity         price  
0                      Aluminum   2432.478261  
1                        Copper   5123.673913  
2                        Nickel  14925.478261  
3  Natural Gas (Henry Hub Spot)      6.890000  
4                      Aluminum   2623.858333  
commodity year_month earnings_date  avg_hourly_earnings     Aluminum  \
0         2006-03-01    2006-03-01                20.71  2432.478261   
1         2006-04-01    2006-04-01                20.83  2623.858333   
2         2006-05-01    2006-05-01                20.73  2852.071429   
3         2006-06-01    2006-06-01                2

In [15]:
idx = df.index  # e.g. RangeIndex

print(idx)                  # prints something like RangeIndex
print("Start:", idx.start)  # start
print("Stop:", idx.stop)    # stop
print("Step:", idx.step)    # step
print("Last valid:", idx.stop - idx.step)  # range

RangeIndex(start=0, stop=944, step=1)
Start: 0
Stop: 944
Step: 1
Last valid: 943


In [10]:
#check for commodity merge dataframe
import pandas as pd

# assume your long form is in df_long and your pivoted wide form is in df_wide
df_long = df.copy()        # the “year_month, earnings_date, …, commodity, price” layout
df_wide = df_pivot.copy()  # the pivoted “year_month, earnings_date, avg_hourly_earnings, Aluminum, Copper, …” layout

# all rows where commodity is Aluminum
aluminum = df_long[df_long['commodity'] == 'Aluminum']


In [ ]:
march_2006 = df_long[df_long['year_month'] == pd.Timestamp('2006-03-01')] # one month check for quality assurance


In [16]:
# Aluminum AND after January 2025
mask = (
    (df_long['commodity'] == 'Aluminum') &
    (df_long['year_month'] <= pd.Timestamp('2025-01-01'))
)
filtered = df_long[mask]
print(filtered)

    year_month earnings_date  avg_hourly_earnings price_date commodity  \
0   2006-03-01    2006-03-01                20.71 2006-03-01  Aluminum   
4   2006-04-01    2006-04-01                20.83 2006-04-01  Aluminum   
8   2006-05-01    2006-05-01                20.73 2006-05-01  Aluminum   
12  2006-06-01    2006-06-01                20.82 2006-06-01  Aluminum   
16  2006-07-01    2006-07-01                20.93 2006-07-01  Aluminum   
..         ...           ...                  ...        ...       ...   
906 2024-09-01    2024-09-01                34.26 2024-09-01  Aluminum   
910 2024-10-01    2024-10-01                34.36 2024-10-01  Aluminum   
914 2024-11-01    2024-11-01                34.47 2024-11-01  Aluminum   
918 2024-12-01    2024-12-01                34.54 2024-12-01  Aluminum   
923 2025-01-01    2025-01-01                34.73 2025-01-01  Aluminum   

           price  
0    2432.478261  
4    2623.858333  
8    2852.071429  
12   2490.954545  
16   2511.833333

In [17]:
idx = df_long.index  # e.g. RangeIndex

print(idx)                  # prints something like RangeIndex
print("Start:", idx.start)  # start
print("Stop:", idx.stop)    # stop
print("Step:", idx.step)    # step
print("Last valid:", idx.stop - idx.step)  # range


RangeIndex(start=0, stop=944, step=1)
Start: 0
Stop: 944
Step: 1
Last valid: 943


In [18]:
# For a datetime index
df = df.set_index('earnings_date')      # or whichever column
print("Full index:", df.index)

print("First index value:", df.index.min())
print("Last index value: ", df.index.max())

# If you just want their integer positions:
positions = df.reset_index().index  # back to RangeIndex
print("First position:", positions.min())
print("Last position: ", positions.max())


Full index: DatetimeIndex(['2006-03-01', '2006-03-01', '2006-03-01', '2006-03-01',
               '2006-04-01', '2006-04-01', '2006-04-01', '2006-04-01',
               '2006-05-01', '2006-05-01',
               ...
               '2025-03-01', '2025-03-01', '2025-03-01', '2025-03-01',
               '2025-04-01', '2025-04-01', '2025-04-01', '2025-04-01',
               '2025-04-01', '2025-05-01'],
              dtype='datetime64[ns]', name='earnings_date', length=944, freq=None)
First index value: 2006-03-01 00:00:00
Last index value:  2025-05-01 00:00:00
First position: 0
Last position:  943


In [19]:
labels = list(df.index)
print("All labels:", labels)

# Or for a quick peek:
print(df.index[:5])   # first 5
print(df.index[-5:])  # last 5


All labels: [Timestamp('2006-03-01 00:00:00'), Timestamp('2006-03-01 00:00:00'), Timestamp('2006-03-01 00:00:00'), Timestamp('2006-03-01 00:00:00'), Timestamp('2006-04-01 00:00:00'), Timestamp('2006-04-01 00:00:00'), Timestamp('2006-04-01 00:00:00'), Timestamp('2006-04-01 00:00:00'), Timestamp('2006-05-01 00:00:00'), Timestamp('2006-05-01 00:00:00'), Timestamp('2006-05-01 00:00:00'), Timestamp('2006-05-01 00:00:00'), Timestamp('2006-06-01 00:00:00'), Timestamp('2006-06-01 00:00:00'), Timestamp('2006-06-01 00:00:00'), Timestamp('2006-06-01 00:00:00'), Timestamp('2006-07-01 00:00:00'), Timestamp('2006-07-01 00:00:00'), Timestamp('2006-07-01 00:00:00'), Timestamp('2006-07-01 00:00:00'), Timestamp('2006-08-01 00:00:00'), Timestamp('2006-08-01 00:00:00'), Timestamp('2006-08-01 00:00:00'), Timestamp('2006-08-01 00:00:00'), Timestamp('2006-09-01 00:00:00'), Timestamp('2006-09-01 00:00:00'), Timestamp('2006-09-01 00:00:00'), Timestamp('2006-09-01 00:00:00'), Timestamp('2006-10-01 00:00:00'), T

In [22]:
import pandas as pd
import numpy as np

# 1. Seed for reproducibility
np.random.seed(42)

# 2. Determine your source of dates
if 'date' in df.columns:
    dates = pd.to_datetime(df['date'])
elif isinstance(df.index, pd.DatetimeIndex):
    dates = df.index.to_series().reset_index(drop=True)
else:
    raise KeyError("DataFrame must have a 'date' column or a DatetimeIndex")

# 3. Simulate one rate per date
rates = np.random.normal(loc=0.20, scale=0.05, size=len(dates))
rates = np.clip(rates, 0.10, 0.30)

# 4. Assemble into a clean, time‐indexed DataFrame
overhead_df = (
    pd.DataFrame({
        'date': dates,
        'overhead_rate_pct': np.round(rates, 4)
    })
    .drop_duplicates('date')     # just in case
    .set_index('date')
    .sort_index()
)

# 5. Preview
print(overhead_df.head())



            overhead_rate_pct
date                         
2006-03-01             0.2248
2006-04-01             0.1883
2006-05-01             0.1765
2006-06-01             0.2121
2006-07-01             0.1494


In [24]:
import pandas as pd

# 1. Identify which date column lives in df
if 'earnings_date' in df.columns:
    date_col = 'earnings_date'
elif 'price_date' in df.columns:
    date_col = 'price_date'
elif 'date' in df.columns:
    date_col = 'date'
else:
    raise KeyError(
        "df must have one of these date columns: "
        "['earnings_date','price_date','date']"
    )

# 2. Normalize df’s date column to datetime
df[date_col] = pd.to_datetime(df[date_col])

# 3. Ensure overhead_df is indexed by a matching DatetimeIndex
if 'date' in overhead_df.columns:
    overhead_df = (
        overhead_df
        .assign(date=pd.to_datetime(overhead_df['date']))
        .drop_duplicates('date')
        .set_index('date')
    )

# 4. Left-merge overhead rates onto df
df_combined = df.merge(
    overhead_df,
    left_on=date_col,
    right_index=True,
    how='left'
)

# 5. (Optional) Tidy up column order
#    e.g. insert 'overhead_rate_pct' immediately after your date_col:
cols = df_combined.columns.tolist()
cols.insert(
    cols.index(date_col) + 1,
    cols.pop(cols.index('overhead_rate_pct'))
)
df_combined = df_combined[cols]

print(df_combined.head())


              year_month  avg_hourly_earnings price_date  overhead_rate_pct  \
earnings_date                                                                 
2006-03-01    2006-03-01                20.71 2006-03-01             0.2248   
2006-03-01    2006-03-01                20.71 2006-03-01             0.2248   
2006-03-01    2006-03-01                20.71 2006-03-01             0.2248   
2006-03-01    2006-03-01                20.71 2006-03-01             0.2248   
2006-04-01    2006-04-01                20.83 2006-04-01             0.1883   

                                  commodity         price  
earnings_date                                              
2006-03-01                         Aluminum   2432.478261  
2006-03-01                           Copper   5123.673913  
2006-03-01                           Nickel  14925.478261  
2006-03-01     Natural Gas (Henry Hub Spot)      6.890000  
2006-04-01                         Aluminum   2623.858333  


In [25]:
# Export to a single-sheet Excel file
df_combined.to_excel('output.xlsx', index=False)


In [26]:
with pd.ExcelWriter('dashboard_data.xlsx', engine='openpyxl') as writer:
    df_long.to_excel(writer, sheet_name='LongForm', index=False)
    df_wide.to_excel(writer, sheet_name='WideForm', index=False)
    df_combined.to_excel(writer, sheet_name='Combined', index=False)